# Feast Basic Customer Transactions Example

This is a minimal example of using Feast. In this example we will
1. Create a synthetic customer feature dataset
2. Register a feature set to represent these features in Feast
3. Ingest these features into Feast
4. Create a feature query and retrieve online feature data
5. Create a feature query and retrieve historical feature data

### 0. Configuration

In [ ]:
import os

# Feast Core acts as the central feature registry
FEAST_CORE_URL = os.getenv('FEAST_CORE_URL', 'core:6565')

# Feast Online Serving allows for the retrieval of real-time feature data
FEAST_ONLINE_SERVING_URL = os.getenv('FEAST_ONLINE_SERVING_URL', 'online-serving:6566')

# Feast Batch Serving allows for the retrieval of historical feature data
FEAST_BATCH_SERVING_URL = os.getenv('FEAST_BATCH_SERVING_URL', 'batch-serving:6567')

# PYTHON_REPOSITORY_PATH is the path to the Python SDK inside the Feast Git Repo
PYTHON_REPOSITORY_PATH = os.getenv('PYTHON_REPOSITORY_PATH', '../../')

### 1. Install Feast SDK

Install from PyPi

In [ ]:
!pip install --ignore-installed --upgrade feast

(Alternative) Install from local repository

In [ ]:
import os
import sys
os.environ['PYTHON_SDK_PATH'] = os.path.join(PYTHON_REPOSITORY_PATH, 'sdk/python')
sys.path.append(os.environ['PYTHON_SDK_PATH'])

In [ ]:
!echo $PYTHON_SDK_PATH

In [ ]:
!python -m pip install --ignore-installed --upgrade -e  ${PYTHON_SDK_PATH}

### 2. Import necessary modules

In [ ]:
import pandas as pd
import numpy as np
from pytz import timezone, utc
from feast import Client, FeatureSet, Entity, ValueType
from feast.serving.ServingService_pb2 import GetOnlineFeaturesRequest
from feast.types.Value_pb2 import Value as Value
from google.protobuf.duration_pb2 import Duration
from datetime import datetime, timedelta
from random import randrange
import random

### 3. Configure Feast services and connect the Feast client

Connect to Feast Core and Feast Online Serving

In [ ]:
client = Client(core_url=FEAST_CORE_URL, serving_url=FEAST_ONLINE_SERVING_URL)

### 4. Create customer features

Here we will create customer features for 5 customers over a month. Each customer will have a set of features for every day.

In [ ]:
days = [datetime.utcnow().replace(hour=0, minute=0, second=0, microsecond=0).replace(tzinfo=utc) \
        - timedelta(day) for day in range(3)][::-1]

customers = [1001, 1002, 1003, 1004, 1005]

In [ ]:
customer_features = pd.DataFrame(
    {
        "datetime": [day for day in days for customer in customers],
        "customer_id": [customer for day in days for customer in customers],
        "daily_transactions": [np.random.rand() * 10 for _ in range(len(days) * len(customers))],
        "total_transactions": [np.random.randint(100) for _ in range(len(days) * len(customers))],
    }
)

print(customer_features.head(500))

### 5. Create feature set for customer features

Now we will create a feature set for these features. Feature sets are essentially a schema that represent
feature values. Feature sets allow Feast to both identify feature values and their structure. The following feature set contains no features yet.

In [ ]:
customer_fs = FeatureSet(
    "customer_transactions",
    entities=[Entity(name='customer_id', dtype=ValueType.INT64)],
    max_age=Duration(seconds=432000)    
)

Here we are automatically inferring the schema from the provided dataset. The two features from the dataset will be added to the feature set

In [ ]:
customer_fs.infer_fields_from_df(customer_features, replace_existing_features=True)

### 6. Register feature set with Feast Core

The apply() method will register the provided feature set with Feast core, allowing users to retrieve features from this feature set

In [ ]:
client.apply(customer_fs)

We test the retrieval of this feature set object (not its data), to ensure that we have the latest version

In [ ]:
customer_fs = client.get_feature_set("customer_transactions")
print(customer_fs)

### 7. Ingest data into Feast for a feature set

In [ ]:
client.ingest("customer_transactions", customer_features)

### 8. Retrieve online features

The process of retrieving features from the online API is very similar to that of the batch API. The only major difference is that users do not have to provide timestamps (only the latest features are returned, as long as they are within the maximum age window)

The example below retrieves online features for a single customer: "1001". It is possible to retrieve any features from feast, even outside of the current project.

In [ ]:
online_features = client.get_online_features(
    feature_refs=[
        f"daily_transactions",
        f"total_transactions",
    ],
    entity_rows=[
        GetOnlineFeaturesRequest.EntityRow(
            fields={
                "customer_id": Value(
                    int64_val=1001)
            }
        )
    ],
)
print(online_features)

### The following section requires Google Cloud Platform (Google Cloud Storage and BigQuery)

### 9. Create a batch retrieval query

In order to retrieve historical feature data, the user must provide an entity_rows dataframe. This dataframe contains a combination of timestamps and entities. In this case, the user must provide both customer_ids and timestamps. 

We will randomly generate timestamps over the last 30 days, and assign customer_ids to them. When these entity rows are sent to the Feast Serving API to retrieve feature values, along with a list of feature ids, Feast is then able to attach the correct feature values to each entity row. 

In [ ]:
event_timestamps = [datetime.utcnow().replace(tzinfo=utc) - timedelta(days=randrange(15), hours=randrange(24), minutes=randrange(60)) for day in range(30)]

entity_rows = pd.DataFrame(
    {
        "datetime": event_timestamps,
        "customer_id": [customers[idx % len(customers)] for idx in range(len(event_timestamps))],
    }
)

print(entity_rows.head(10))

### 10. Retrieve historical/batch features

Next we will create a new client object, but this time we will configure it to connect to the Batch Serving Service. This service will allow us to retrieve historical feature data.

In [ ]:
batch_client = Client(core_url=FEAST_CORE_URL, serving_url=FEAST_BATCH_SERVING_URL)
batch_client.set_project("customer_project")

By calling the `get_batch_features` method we are able to retrieve a `job` object for the exporting of feature data. For every entity and timestamp combination in `entity_rows` we will be receiving a row with feature values joined to it.

In [ ]:
job = batch_client.get_batch_features(
                            feature_refs=[
                                f"customer_project/daily_transactions", 
                                f"customer_project/total_transactions", 
                               ],
                            entity_rows=entity_rows
                         )

Once the job is complete, it is possible to retrieve the exported data (from Google Cloud Storage) and load it into memory as a Pandas Dataframe.

In [ ]:
df = job.to_dataframe()
print(df.head(10))